In [1]:
# import sys
# !{sys.executable} -m pip install gumpy
# !{sys.executable} -m pip install piezo

In [1]:
import gumpy, piezo, numpy, copy, random, pandas as pd

In [2]:
gene = 'pncA'
drug = 'PZA'
catalogue_file = 'NC_000962.3_WHO-UCN-GTB-PCI-2021.7_v1.0_GARC1_RUS.csv'

Some lookup `dict` and `list`s we will need later to translate from codons to amino acids and vice versa

In [4]:
aminoacids = 'FFLLSSSSYY!!CC!WLLLLPPPPHHQQRRRRIIIMTTTTNNKKSSRRVVVVAAAADDEEGGGG'
bases = ['t', 'c', 'a', 'g']
all_codons = numpy.array([a+b+c for a in bases for b in bases for c in bases])
codon_to_amino_acid = dict(zip(all_codons, aminoacids))
amino_acid_to_codon={}
for i,j in zip(aminoacids, all_codons):
    if i in amino_acid_to_codon:
        amino_acid_to_codon[i].append(j)
    else:
        amino_acid_to_codon[i] = [j]

Here we are using a supplied catalogue to define the mutations associated with resistance

In [3]:
catalogue = piezo.ResistanceCatalogue(catalogue_file)

specific_resistance_mutations = catalogue.catalogue.rules[ (catalogue.catalogue.rules.PREDICTION=='R') &\
                                                          (catalogue.catalogue.rules.MUTATION_TYPE=='SNP') &\
                                                          (catalogue.catalogue.rules.MUTATION_AFFECTS=='CDS') &\
                                                          (catalogue.catalogue.rules.MUTATION.str[-1]!='!') &\
                                                          (catalogue.catalogue.rules.POSITION!='*')]

pnca_resistant_mutations = specific_resistance_mutations[(specific_resistance_mutations.DRUG==drug) & (specific_resistance_mutations.GENE==gene)]

In [4]:
print("For pncA there are %i mutations associated with resistance to PZA" % len(pnca_resistant_mutations) )

For pncA there are 154 mutations associated with resistance to PZA


..where as for the susceptible mutations we are simply randomly choosing a 1SNP amino acid mutation that results in a missense mutation anywhere in the protein (except at the codons where we are introducing resistance)

In [5]:
#* PARAMS

genbank_file = 'NC_000962.3.gbk'
n_samples = 10
proportion_resistant = 0.5
resistant_mutations = list(pnca_resistant_mutations.MUTATION)
susceptible_mutations = 'uniform'
n_res = 1
n_sus = 1
output = 'mutations' 
debug = False

Now let's build the `Genome` object so we can access the genes later

In [8]:
reference = gumpy.Genome(genbank_file)

We are only dealing with one gene here

In [9]:
reference_gene = reference.build_gene(gene)

Because we want to produce `n_samples` we have to wrap it all in a big `for` loop...

In [10]:
for n_sample in range(n_samples):

    # let's take a deepcopy so we don't accidentally alter the reference
    sample_gene = copy.deepcopy(reference_gene)

    #* WORK OUT RESISTANT MUTATIONS
    if random.random() < proportion_resistant:
        label='R'
        if debug:
            print('Resistant Sample!')
        # assuming that a Poisson distribution describes the expected number
        number_resistant = numpy.random.poisson(n_res)
    else:
        label='S'
        if debug:
            print('Susceptible Sample!')
        number_resistant = 0

    # choose the resistant mutations we want to incorporate
    selected_resistant_mutations = random.choices(resistant_mutations, k=number_resistant)

    if debug:
        print("R: ", selected_resistant_mutations)

    # first, identify the codons being mutated as we will want to avoid these for susceptible mutations
    positions_altered = []
    for mutation in selected_resistant_mutations:
        aa_pos = int(mutation[1:-1])
        positions_altered.append(aa_pos)

    # Get amino acid positions that are not altered by selected resistant mutations
    remaining_aa_positions = sample_gene.amino_acid_number[~numpy.isin(sample_gene.amino_acid_number, positions_altered)]

    #* WORK OUT SUSCEPTIBLE MUTATIONS
    number_susceptible = numpy.random.poisson(n_sus)

    selected_susceptible_mutations = []

    # now randomly choose some susceptible mutations (i.e. "uniformly")
    for susceptible_codon in random.choices(remaining_aa_positions, k=number_susceptible):
        
        ref_codon = sample_gene.codons[sample_gene.amino_acid_number==susceptible_codon][0]
        ref_aa = codon_to_amino_acid[ref_codon]

        possible_alt_aa = []

        for alt_codon in codon_to_amino_acid:

            # no synoymous mutations
            if codon_to_amino_acid[alt_codon] != ref_aa:
                possible_aa = codon_to_amino_acid[alt_codon]

                # no premature Stop codons -- may want to make this a parameter in future
                if possible_aa != "!":                
                    n_snps = sum(1 for a, b in zip(ref_codon, alt_codon) if a != b)

                    # only look for SNPs and mutations not in our list of resistance associated mutations
                    if n_snps == 1 and possible_aa not in resistant_mutations:
                        possible_alt_aa.append(possible_aa)

        if debug:
            print('Possible alternate amino acid for susceptible mutation:', possible_alt_aa)

        alt_aa = random.choice(possible_alt_aa)

        alt_mutation = ref_aa + str(susceptible_codon) + alt_aa

        selected_susceptible_mutations.append(alt_mutation)

    if debug:
        print("S: ", selected_susceptible_mutations)

    selected_mutations = selected_resistant_mutations + selected_susceptible_mutations


    #* GET MUTATIONS FOR SAMPLE GENE
    for mutation in selected_mutations:

        ref_aa = mutation[0]
        alt_aa = mutation[-1]
        aa_pos = int(mutation[1:-1])

        if debug:
            print(mutation)

        ref_codon = sample_gene.codons[sample_gene.amino_acid_number==aa_pos][0]
    
    #* GET BASE CHANGES
        alt_codon=None
        for codon in amino_acid_to_codon[alt_aa]:
            counter = sum(1 for a, b in zip(ref_codon, codon) if a != b)
            if counter==1:
                alt_codon = codon
                break

        base_pos = 3*aa_pos -2
        for i,j in zip(ref_codon, alt_codon):
            if i!=j:
                ref_base = i
                alt_base = j
                break
            base_pos+=1

        assert reference_gene.nucleotide_sequence[reference_gene.nucleotide_number==base_pos][0] == ref_base

        sample_gene.nucleotide_sequence[sample_gene.nucleotide_number==base_pos] = alt_base

    #* RETURN (PRINT) EITHER MUTATIONS OR MUTATED ALLELE    
    sample_gene._translate_sequence()
    print("SAMPLE %i, LABEL %s, %i resistant mutations, %i susceptible mutations" % (n_sample, label, number_resistant, number_susceptible))
    if output == 'allele':
        sample_amino_acid_sequence = ''.join(i for i in sample_gene.amino_acid_sequence)
        print(sample_amino_acid_sequence)
    elif output == 'mutations':
        diff = reference_gene - sample_gene
        for i in diff.mutations:
            print(i)
    else:
        raise ValueError('output can only be one of allele or mutations!')
    
    # break


SAMPLE 0, LABEL S, 0 resistant mutations, 1 susceptible mutations
N149K
SAMPLE 1, LABEL S, 0 resistant mutations, 1 susceptible mutations
L182W
SAMPLE 2, LABEL S, 0 resistant mutations, 0 susceptible mutations
SAMPLE 3, LABEL S, 0 resistant mutations, 1 susceptible mutations
T153S
SAMPLE 4, LABEL S, 0 resistant mutations, 2 susceptible mutations
D12V
H137N
SAMPLE 5, LABEL S, 0 resistant mutations, 1 susceptible mutations
D33A
SAMPLE 6, LABEL R, 0 resistant mutations, 0 susceptible mutations
SAMPLE 7, LABEL S, 0 resistant mutations, 1 susceptible mutations
K48N
SAMPLE 8, LABEL S, 0 resistant mutations, 0 susceptible mutations
SAMPLE 9, LABEL R, 2 resistant mutations, 1 susceptible mutations
D12E
R140C
V180F


In [6]:
from src.sbmlsim import sbmlsim

gene = 'pncA'
drug = 'PZA'
catalogue_file = 'NC_000962.3_WHO-UCN-GTB-PCI-2021.7_v1.0_GARC1_RUS.csv'
genbank_file = 'NC_000962.3.gbk'

batch = sbmlsim.batch(gene, drug, catalogue_file, genbank_file)

In [7]:
n_samples = 10
n_res = 1
n_sus=1
proportion_resistant = 0.5

alleles, muts = batch.generate_batch(n_samples, proportion_resistant, n_res, n_sus, distribution = "poisson")

In [3]:
display(alleles)
display(muts)

,allele,label,num_res_mutations,num_sus_mutations
sample,,,,
0,MRALIIADVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,R,1,0
1,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,R,1,0
2,MRALITVDVQNDFCAGGSLAVTGGAALARAISDYLAEAADYHHVVA...,R,2,2
3,MRALIIVDVQNDFCEGGSLVVTGGAALARAISDYLAEAADYHHVVA...,R,0,1
4,MRALIIVDVQNDFCEGGSLAVTGGAALARASSDYLAEAADYHHVVA...,R,2,1
5,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,R,1,1
6,MRASIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,R,1,0
7,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,S,0,2
8,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,S,0,0


mutation_label  gene
sample mutation                     
0      V7A                   R  pncA
1      F81V                  R  pncA
2      I6T                   R  pncA
       E15A                  S  pncA
       A102P                 R  pncA
       G113A                 S  pncA
3      A20V                  S  pncA
4      I31S                  R  pncA
       L117P                 S  pncA
       T135N                 R  pncA
5      S104R                 R  pncA
       A134G                 S  pncA
6      L4S                   R  pncA
7      H82L                  S  pncA
       M175R                 S  pncA
9      I5L                   S  pncA
       L159V                 S  pncA

In [ ]:
# STOP codons in susceptible mutations ??

In [ ]:
# sample | allele | label | num_res_mutations | res_mutations (list) | num_sus_mutations | sus_mutations (list)

In [8]:
df = pd.DataFrame(columns=['sample', 'allele', 'label', 'num_res_mutations', 'res_mutations', 'num_sus_mutations', 'sus_mutations']).set_index('sample')

In [9]:
df.loc[1] = ['MRALIIVDVQNAFCEGGSLAVTGGAALARAISDYLAEAADYHHVVATKDFHIDPGDHFSGTPDYSSSWPPHCVSGTPGADFHPSLDTSAIEAGFYKGAYTGAYSGFEGVDENGTPLLNWLRQRGVDEVDVVGIATDHCVRQTAEDAVRNGLATRVLVDLTAGVSADTTVAALEEMRTASVELVCSS', 'S', 0, [], 1, ['ATG']]

In [10]:
df

,allele,label,num_res_mutations,res_mutations,num_sus_mutations,sus_mutations
sample,,,,,,
1,MRALIIVDVQNAFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,S,0,[],1,[ATG]


In [3]:
# import pandas as pd
# mutations_df = pd.DataFrame(
#             columns=[
#                 "sample",
#                 "mutation",
#                 "mutation_label",
#                 "gene"
#             ]
#         ).set_index(["sample", "mutation"]) #? multiindex correct here?

In [6]:
# mutations_df.loc[(0, 'T167S' ), ['mutation_label', 'gene']] = ['s', 'pncA']

In [32]:
# mutations_df

,,mutation_label,gene
sample,mutation,,
1,G133S,s,pncA
